# PWV05: Fit Gaussian process and linear fit on PWV time sequence and Fetch Spectra and save in hdf5

- author Sylvie Dagoret-Campagne
- creation date 2025-09-16 : version v1
- affiliation : IJCLab
- Kernel @usdf **w_2025_36*
- Home emac : base (conda)
- laptop : conda_py310

**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

#### Create output directory for figures and data

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV05_HoloFITLinearandGPPWVandFetchNightSpectra"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
pathdata = "data_PWV05_HoloFITLinearandGPPWVandFetchNightSpectra"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datapath_output = os.path.join(pathdata,"pernightspectra") 
if not os.path.exists(datapath_output):
    os.makedirs(datapath_output) 

#### Imports

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

import numpy as np
from numpy.linalg import inv


from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

import scipy
from scipy.optimize import curve_fit,least_squares
from scipy.interpolate import RegularGridInterpolator
from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

# new color correction model
import pickle

from tqdm import tqdm

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
import spectractor
from spectractor.tools import from_lambda_to_colormap, wavelength_to_rgb
from spectractor.simulation.throughput import load_transmission,plot_transmission_simple,TelescopeTransmission
from spectractor import parameters
from spectractor.extractor import dispersers
from spectractor.config import load_config,set_logger
import os,sys,re

In [ ]:
import getCalspec.getCalspec as gcal

#### Butler access

- these access may change with time
- check my collections are found

In [ ]:
def get_whereclause(expos_id):
    return f"exposure.id = {expos_id} and instrument = \'LATISS\'"

In [ ]:
from lsst.summit.utils.utils import checkStackSetup
checkStackSetup()

In [ ]:
# Notice before I had to use these lines to access to butler
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

# Now I have to use these lines
import lsst.daf.butler as dafButler
repo = "/repo/main"
#repo = "/repo/embargo_old"


butler = dafButler.Butler(repo)
registry = butler.registry

for c in sorted(registry.queryCollections()):
    #if "u/jneveu" in c and "auxtel_atmo" in c and "SensorFlat" in c and "FixA1" in c:
    if "u/dagoret" in c and "auxtel_atmo" in c and "SensorFlat" in c and "FixA1" in c:
        print(c)

#### Environnement for rubinsimphot
- Configure to access to rubinsimphot
- Note it is important to have the rubinsimphot path in the python path
- check the dm_version repository the newpythonpath 

In [ ]:
dm_version="repos_w_2025_36" 
machine_name = os.uname().nodename
print(machine_name)
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    #machine_name_notebook platform = 'dagoret-nb'
    print("Set environement for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),f"repos/{dm_version}/rubinsimphot/src")
    sys.path.append(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)   
elif "dagoret-nb" in machine_name:
    print("Set environement for Rubin Platform at  USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),f"repos/{dm_version}/rubinsimphot/src")
    sys.path.append(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environnement named conda_py310")
else:
    print("Your current machine name is {machine_name}. Check your python environment")

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data.data_sets import  get_data_dir

#README.md        darksky.dat      filter_r.dat     hardware_g.dat   hardware_y.dat   lens3.dat        total_g.dat      total_y.dat
#README_SOURCE.md detector.dat     filter_u.dat     hardware_i.dat   hardware_z.dat   m1.dat           total_i.dat      total_z.dat
#atmos_10.dat     filter_g.dat     filter_y.dat     hardware_r.dat   lens1.dat        m2.dat           total_r.dat      version_info
#atmos_std.dat    filter_i.dat     filter_z.dat     hardware_u.dat   lens2.dat        m3.dat           total_u.dat
hardware_filenames = ["hardware_u.dat","hardware_g.dat","hardware_r.dat","hardware_i.dat","hardware_z.dat","hardware_y.dat"] 
filter_filenames = ["filter_u.dat","filter_g.dat","filter_r.dat","filter_i.dat","filter_z.dat","filter_y.dat" ]
total_filenames = ["total_u.dat","total_g.dat","total_r.dat","total_i.dat","total_z.dat","total_y.dat" ]
filter_tagnames = ["u","g","r","i","z","y"]
Filter_tagnames = ["U","G","R","I","Z","Y"]
filtercolor_tagnames = ["u-g","g-r","r-i","i-z","z-y"]
Filtercolor_tagnames = ["U-G","G-R","R-I","I-Y","Z-Y"]
filter_color = ["b","g","r","orange","grey","k"]
NFILT=len(filter_filenames)

WLMIN=300.
WLMAX=1100.
WLBIN=1.
NWLBIN=int((WLMAX-WLMIN)/WLBIN)
WL=np.linspace(WLMIN,WLMAX,NWLBIN)

In [ ]:
#FILTERWL: precalculated array containing center, boundaries and width of each filter.
#index 0 : minimum wavelength of filter border
#index 1 : minimum wavelength of filter border
#index 2 : center wavelength of filter
#index 3 : filter width


FILTERWL = np.array([[ 324.03003755,  402.12765957,  363.59690349,   78.09762203],
       [ 392.11514393,  561.32665832,  473.54069923,  169.21151439],
       [ 542.3028786 ,  700.50062578,  619.49926767,  158.19774718],
       [ 681.47684606,  827.65957447,  752.01084117,  146.18272841],
       [ 808.63579474,  932.79098874,  868.488419  ,  124.15519399],
       [ 914.76846058, 1044.93116395,  969.10570859,  130.16270338]])

FILTERWL_auxtel = np.array([[ 352.7 ,  395.9 ,  374.3 ,   43.2 ],
                     [ 387.6 ,  566.2 ,  476.9 ,  178.6 ],
                     [ 541.4 ,  715.5 ,  628.45,  174.1 ],
                     [ 673.3 ,  870.9 ,  772.1 ,  197.6 ],
                     [ 805.6 , 1090.7 ,  948.15,  285.1 ]])


F0 = 3631.0 # Jy 1, Jy = 10^{-23} erg.cm^{-2}.s^{-1}.Hz^{-1}
Jy_to_ergcmm2sm1hzm1 = 1e-23
DT = 30.0 # seconds
gel = 1.1
#hP = 6.62607015E-34 # J⋅Hz−1
hP = 6.626196E-27
A  = np.pi*642.3**2 # cm2  Reff=6.423 m
A_auxtel  = 9636.0 # cm2

#ZPT_cont =  2.5 \log_{10} \left(\frac{F_0 A \Delta T}{g_{el} h} \right)
ZPTconst = 2.5*np.log10(F0*Jy_to_ergcmm2sm1hzm1*A*DT/gel/hP)

In [ ]:
fdir = get_data_dir()
bandpass_inst = {}
path_rubin_sim_throughput = os.path.join(fdir, 'throughputs', 'baseline')
for index,filename in enumerate(hardware_filenames):
    fullfilename=os.path.join(path_rubin_sim_throughput,filename)
    arr= np.loadtxt(fullfilename)
    # interpolate  filter transmission
    ff = interpolate.interp1d(x=arr[:,0], y=arr[:,1],fill_value="extrapolate")
    fname = filter_tagnames[index]
    bandpass_inst[fname] = Bandpass(wavelen=WL,sb=ff(WL))

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(WL,bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(WL,bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total Rubin-LSST filter throughput")
plt.show()

## Access to throughput

- Please check the curve used for the Throughput
- The version of throughput is defined in ~/rubin-user/holo_atmo_2024-10-16 /processStar.yaml
as 'instrumentTransmissionOverride': 'multispectra_holo4_003_HD142331_20230802_348-594_HD146233_AuxTel_v3
.1.0_doSensorFlat_FreePressure_BG40ReScaled1.09_throughput.txt'

In [ ]:
spectractor_path = spectractor.__path__[0]
spectractor_path

In [ ]:
#config_path = "~/repos/repos_w_2023_44/Spectractor/config/auxtel.ini"
config_path = f"/home/d/dagoret/repos/repos_w_2025_36/Spectractor/config/auxtel.ini"
config_fullpath = os.path.join(spectractor.__path__[0],"../config/auxtel.ini")
throughput_path = "/home/d/dagoret/repos/repos_w_2025_36/Spectractor/spectractor/simulation/AuxTelThroughput"
#throughput_filename="multispectra_holo4_003_HD142331_20230802_AuxTel_doGainsPTC_v3.0.3_throughput.txt"
throughput_filename="multispectra_holo4_003_HD142331_20230802_348-594_HD146233_AuxTel_v3.1.0_doSensorFlat_FreePressure_BG40ReScaled1.09_throughput.txt"

In [ ]:
throughput_fullfilename = os.path.join(throughput_path,throughput_filename)

In [ ]:
#tel_lambdas, tel_thr, tel_errthr = load_transmission(throughput_fullfilename) 
tel_arr = np.loadtxt(throughput_fullfilename)
tel_lambdas = tel_arr[:,0] 
tel_thr = tel_arr[:,1]
tel_errthr = tel_arr[:,2]

class Telescope():
    tel_lambdas = tel_lambdas 
    tel_thr  = tel_thr 
    tel_errthr = tel_errthr 
    func = interpolate.interp1d(tel_lambdas, tel_thr, kind='linear',  bounds_error=False, fill_value=0, 
                                    assume_sorted=False)
    @classmethod
    def transmission(cls, wls):
        return  cls.func(wls)         

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot()
ax.plot(tel_lambdas,tel_thr,'r-') 
ax.fill_between(tel_lambdas, y1=tel_thr-tel_errthr , y2=tel_thr+tel_errthr,facecolor="grey")
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("Throughput")
ax.set_title("Auxtel Telescope throughput")
plt.show()

In [ ]:
plt.plot(WL,Telescope.transmission(WL))

#### Hologram throughput

In [ ]:
holo_fullfilename  ="/home/d/dagoret/repos/repos_w_2025_36/Spectractor/spectractor/extractor/dispersers/holo4_003/transmission.txt"

In [ ]:
holo_arr = np.loadtxt(holo_fullfilename)
holo_lambdas = holo_arr[:,0] 
holo_thr = holo_arr[:,1]
holo_errthr = holo_arr[:,2]

class Disp():
    holo_lambdas = holo_lambdas 
    holo_thr  = holo_thr 
    holo_errthr = holo_errthr 
    func = interpolate.interp1d(holo_lambdas, holo_thr, kind='linear',  bounds_error=False, fill_value=0, 
                                    assume_sorted=False)
    @classmethod
    def transmission(cls, wls):
        return  cls.func(wls)       

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot()
ax.plot(holo_lambdas,holo_thr,'r-') 
ax.fill_between(holo_lambdas, y1=holo_thr-holo_errthr , y2=holo_thr+holo_errthr,facecolor="grey")
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("Transmission")
ax.set_title("Hologram transmission")
plt.show()

In [ ]:
plt.plot(WL,Disp.transmission(WL))

### Load my stuff

In [ ]:
sys.path.append("../lib")
from libauxtelspectra import *
from libanaspectra import *
from config_spectractor import *

In [ ]:
#config_fullpath = os.path.join(spectractor.__path__[0],"../config/auxtel.ini")
#load_config_local(config_fullpath)

In [ ]:
parameters.OBS_NAME

## getObsAtmo emulator

In [ ]:
from getObsAtmo.getObsAtmo import ObsAtmo
emul=  ObsAtmo(obs_str= "LSST")

## Access to spectra

- But here we don't want to get all spectra. We fetch the sepected ones later

In [ ]:
# my run 2024-09-24
#my_collection = ['u/dagoret/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder/20240924T161119Z']
# my run 2024-10-16
#my_collection = ['u/dagoret/auxtel_atmosphere_202209_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip/20241016T184601Z']
my_collection = ['u/dagoret/auxtel_run_20250912a']
datasetRefs = registry.queryDatasets(datasetType='spectractorSpectrum', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
where = "instrument='LATISS'" 
records = list(butler.registry.queryDimensionRecords('visit', datasets='spectractorSpectrum', where=where,  collections=my_collection))

refs = list(set(butler.registry.queryDatasets('spectractorSpectrum',  where=where,  collections=my_collection)))
# records = list(butler.registry.queryDimensionRecords('visit', datasets='raw', where=where))

In [ ]:
print(len(records))
records[0]

In [ ]:
def GetListOfSpectra(butler,list_of_visits,the_collection):
    """
    Retrieve a list of Spectra:
    Parameters:
      butler : the butler
      list_of_visits : the list of visit id
      collection : the collection

    Returns:
     all_params_spectrum = []
     all_params_spectrogram = []
     all_visitid = []
     all_headers = []
     all_spectra = []
    """


    # container contining the data
    all_params_spectrum = []
    all_params_spectrogram = []
    all_visitid = []
    all_headers = []
    all_spectra = []

    # loop on visits
    for idx, visitid in enumerate(list_of_visits):
        try:         
            spec =  butler.get('spectractorSpectrum', visit=visitid, collections=the_collection, detector=0, instrument='LATISS')
            all_headers.append(spec.header)
            all_spectra.append(spec)
            p = butler.get('spectrumLibradtranFitParameters', visit=visitid, collections=the_collection, detector=0, instrument='LATISS')
            all_params_spectrum.append(p)
            p = butler.get('spectrogramLibradtranFitParameters', visit=visitid, collections=the_collection, detector=0, instrument='LATISS')
            all_params_spectrogram.append(p)
            all_visitid.append(visitid)
        #except ValueError:
        except Exception as inst:
            except_type = type(inst)
            except_args = inst.args
            print("catch exception ", inst, "type =",except_type, "args = ",except_args) 
            print("\t >>>>> Skip visitid ", visitid)
            continue
    return all_visitid, all_headers, all_params_spectrum , all_params_spectrogram, all_spectra
    


## Functions

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = True
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
version_results = "v1"
legendtag = {"v1" : "v3.1.0"}

In [ ]:
atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",}


In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
list(df_spec.columns)

### Remove spectra with red filter

In [ ]:
df_spec['FILTER'].unique()

In [ ]:
FLAG_REMOVE_FILTERS = True
if FLAG_REMOVE_FILTERS:
    df_spec=df_spec[df_spec["FILTER"] == 'empty']
    df_spec.reset_index(inplace=True)  

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

#### UTC

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
df_spec["dt_midnight"].hist(bins=48,range=(-12,12),ax=ax,facecolor="blue") 
ax.set_xlabel("time relative to midnight (hour)")
ax.set_title("Observation time")

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="PShP",ax=ax,marker=".",c="b",lw=0.5,label="Merra2",ms=1,legend=leg)   
df_spec.plot(x="Time",y="OUTPRESS",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("OUTPRESS")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Pressure (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


#ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pressure_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX)   
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo vs time (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pwv_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


## Apply Quality selection cuts

In [ ]:
def getSelectionCutOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
def getSelectionCutNoPolarOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] != "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolarOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCut(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) 
    #(df_spec['EXPTIME'] > 20.) 
    #& (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCutNoPolar(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) & \
    (df_spec["TARGET"] != "HD185975")
    #(df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolar(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) & \
    (df_spec["TARGET"] == "HD185975")
    #(df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

## Recompute night boundaries

In [ ]:
#dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()

#ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=0.5,ms=1,label="Merra2",legend=leg)     
    
df_spec_sel[df_spec_sel.Year==2022].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="g",lw=0.0,grid=True,label="2022",legend=leg,ms=5,alpha=1.)
df_spec_sel[df_spec_sel.Year==2023].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="b",lw=0.0,grid=True,label="2023",legend=leg,ms=5,alpha=1.)
df_spec_sel[df_spec_sel.Year==2024].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label="2024",legend=leg,ms=5,alpha=1.)
#df_spec_np_f.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='o',c="grey",lw=0.0,grid=True,label="faint,no polar",legend=leg,ms=5,alpha=0.2)
#df_spec_np_b.plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='o',c="r",lw=0.0,grid=True,label="bright,no polar",legend=leg,ms=5,alpha=0.2)
#df_spec_np.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='o',c="m",lw=0.0,grid=True,label="no polar",legend=leg,ms=5,alpha=0.5)
#df_spec_wp.plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='o',c="g",lw=0.0,grid=True,label="with polar",legend=leg,ms=5,alpha=0.5)
ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date (since January)")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo (modulo 1 year)",fontweight="bold")
ax.legend()

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,15.)
#ax.set_xlim(TMIN,TMAX)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights_modulo1year"+figtype
plt.savefig(figname)
plt.show()

## Plot night by night

In [ ]:
def ComputeRepeatability(df):
    """
    Compute Repeatability of PWV for Spectrogram and and Spectrum
    
    """
    N = len(df)
    dfout = pd.DataFrame(index=df.index,columns = ["nightObs","dt","dt_rep","dPWVx_rep","dPWVy_rep","targflag_rep","Npoints"])
    dfout["targflag_rep"].astype(bool)
    #dfout["Npoints"].astype(int)
    
    target_old = "No"
    time_old = 0.
    pwvx_old = 0.
    pwvy_old = 0.
    
    for index in range(N):
        
        nightObs =  df.iloc[index]["nightObs"]
        
        if index ==0:
            dt0 = df.iloc[index]["dt"]
            dfout.iloc[index] = [ nightObs,dt0, 0., 0., 0., False,N]
        else:
            target_new = df.iloc[index]["TARGET"]
            time_new = df.iloc[index]["dt"]
            pwvx_new = df.iloc[index]["PWV [mm]_x"]
            pwvy_new = df.iloc[index]["PWV [mm]_y"]
            
            flag_target = (target_new == target_old)
            dPWVx_rep = pwvx_new - pwvx_old
            dPWVy_rep = pwvy_new - pwvy_old
            dt_rep = (time_new-time_old)*3600. # in seconds
            
            dfout.iloc[index] = [ nightObs,time_new, dt_rep, dPWVx_rep, dPWVy_rep, flag_target,N]
        
        target_old = df.iloc[index]["TARGET"]
        time_old = df.iloc[index]["dt"]
        pwvx_old = df.iloc[index]["PWV [mm]_x"]
        pwvy_old = df.iloc[index]["PWV [mm]_y"]
        
    return dfout

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

## Fits gaussien et lineaires

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt_midnight"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

In [ ]:
#
# Gaussian Process regression
# https://scikit-learn.org/1.5/auto_examples/gaussian_process/plot_gpr_noisy_targets.html
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [ ]:
def MakeGaussianProcessFit(df_night_pwv_curve):
    """
    """

    x_train = df_night_pwv_curve["dt_midnight"].values
    xmin =  x_train.min()
    xmax = x_train.max()
    xfit = np.linspace(xmin,xmax,50)
    X_fit =xfit.reshape(-1,1)
    X_train = x_train.reshape(-1, 1)
    
    y_train = df_night_pwv_curve["PWV [mm]_x"].values
    y_mean = y_train.mean()
    yerr_train = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y_train)

    noise_std= 0.5
    
    kernel = 1. * RBF(length_scale=5.0, length_scale_bounds=(0.5, 12.))
    gaussian_process = GaussianProcessRegressor(kernel=kernel,alpha=noise_std**2 ,n_restarts_optimizer=9)
    
    gaussian_process.fit(X_train, y_train)

    mean_prediction, std_prediction = gaussian_process.predict(X_fit, return_std=True)
    return xfit, mean_prediction, std_prediction, gaussian_process

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# https://scikit-learn.org/1.5/auto_examples/linear_model/plot_ard.html
from sklearn.linear_model import ARDRegression,BayesianRidge

In [ ]:
POLY_DEGREE_MAX = 10

In [ ]:
def MakeLinearModelFit(df_night_pwv_curve,degree_max = POLY_DEGREE_MAX ):
    """
    https://scikit-learn.org/1.5/auto_examples/linear_model/plot_ard.html
    ARDRegression and BayesianRidge
    """

    x_train = df_night_pwv_curve["dt_midnight"].values
    xmin =  x_train.min()
    xmax = x_train.max()
    xfit = np.linspace(xmin,xmax,50)
    X_fit =xfit.reshape(-1,1)
    X_train = x_train.reshape(-1, 1)
    
    y_train = df_night_pwv_curve["PWV [mm]_x"].values
    y_mean = y_train.mean()
    yerr_train = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y_train)

    ard_poly = make_pipeline(
    PolynomialFeatures(degree=degree_max, include_bias=False),StandardScaler(),ARDRegression(),).fit(X_train, y_train)
    
    brr_poly = make_pipeline(
    PolynomialFeatures(degree=degree_max, include_bias=False),StandardScaler(),BayesianRidge(),).fit(X_train, y_train)

    y_ard, y_ard_std = ard_poly.predict(X_fit, return_std=True)
    y_brr, y_brr_std = brr_poly.predict(X_fit, return_std=True)

    y_ard_pred = ard_poly.predict(X_train,return_std=False)
    resy = y_train - y_ard_pred
    
    return xfit,y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean

In [ ]:
# Container of images that all goes in a pdf
all_figs_to_pdf = []

In [ ]:
all_dateObs_sel = {}

# loop on nights
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_spec_night_wp = df_spec_wp[df_spec_wp["nightObs"] == night]
    df_spec_night_np = df_spec_np[df_spec_np["nightObs"] == night]
    df_spec_night_np_f = df_spec_np_f[df_spec_np_f["nightObs"] == night]
    df_spec_night_np_b = df_spec_np_b[df_spec_np_b["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_wp = df_spec_night_wp[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_b = df_spec_night_np_b[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_f = df_spec_night_np_f[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np = df_spec_night_np[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    list_of_targets = df_night_pwv_curve["TARGET"].unique()
    str_list_of_targets = "\n".join(list_of_targets)
     
    # convert in hours
    #df_night_pwv_curve["dt"] = (pd.to_datetime(df_night_pwv_curve["Time"]) - tmin).dt.total_seconds()/3600.   
    #df_night_pwv_curve["dt"] = df_night_pwv_curve.apply(lambda row : (pd.to_datetime(row["Time"])-tmin).seconds,axis=1)
    df_night_pwv_curve.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_wp.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_f.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_b.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    #date_form = DateFormatter("%y-%m-%dT%H")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    N= len(df_night_pwv_curve)

    if N>10:
        # does the fit
        
        #x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
            
        xfit, mean_prediction, std_prediction, gp = MakeGaussianProcessFit(df_night_pwv_curve)
        xfit2, y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean = MakeLinearModelFit(df_night_pwv_curve)
        
        #textstr2 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",
        #             f"chi2/ndeg : {chi2dof:.2f}",       
        #            ))

        #textstr3 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",  
        #            ))
    

        #stat.loc["slope","PWV [mm]_x"] = slope 
        #stat.loc["slope_err","PWV [mm]_x"] = slope_err 
        #stat.loc["chi2","PWV [mm]_x"] = chi2dof
        
        # plot
           
        fig = plt.figure(figsize=(16,10),constrained_layout=True)
        gs = GridSpec(2, 2,  width_ratios=[1,1],figure=fig)
    
        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
        
        leg=ax.get_legend()
        leg2=ax2.get_legend()
      
        
        ################################
        # left figure : Gaussian Process
        ################################
        
        ax.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",ms=1)
        df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="polar",ms=6,alpha=1)
        df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="bright, no polar",ms=6,alpha=1)
        df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="faint, no polar",ms=6,alpha=1)

        # fit result
        ax.plot(xfit, mean_prediction,"b-",lw=3,label="Gaussian Process (RBF)")
        ax.fill_between(xfit,mean_prediction - 1.96 * std_prediction,mean_prediction + 1.96 * std_prediction,alpha=0.3,label=r"95% CL",facecolor='grey')
        
        ax.set_xlabel("hours")
        ax.set_ylim(0.,15.)
        ax.legend(loc="upper right")
        #ax.plot(xfit,yfit,"k-")
        #ax2.text(0.05, 0.95, textstr2, transform=ax2.transAxes, fontsize=14,
        #verticalalignment='top', bbox=props)
        ax.set_title(f"GP fit for night {night}")
        ax.axvline(x=0,color="k",lw=2)


        ############################
        # right figure : linear fits
        ############################
        ax2.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",ms=1)
        df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="polar",ms=6,alpha=1)
        df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="bright, no polar",ms=6,alpha=1)
        df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="faint, no polar",ms=6,alpha=1)


        ax2.plot(xfit2, y_ard,"g-",lw=3,label="ARDRegression")
        ax2.fill_between(xfit2, y_ard - 1.96 * y_ard_std,y_ard + 1.96 * y_ard_std,alpha=0.3,label=r"95% CL",facecolor='green')

        #ax2.plot(xfit2, y_brr,"r-",label="BayesianRidge")
        #ax2.fill_between(xfit2, y_brr - 1.96 * y_brr_std,y_brr + 1.96 * y_brr_std,alpha=0.5,label=r"95% CL",facecolor='red')

        
        ax2.set_title(f"Linear fit for night {night}")
        ax2.set_xlabel("hours")
        ax2.set_ylim(0.,15.)
        ax2.legend()
        ax2.axvline(x=0,color="k",lw=2)
        
        #fig.tight_layout()
        #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

        all_figs_to_pdf.append(fig) 

        figname =f"{pathfigs}/gp_ard_pwv_per_night_{night}"+figtype
        plt.savefig(figname)
        plt.show()

        # add statistics
        all_dateObs_sel[night] = stat

    
        

## Plot night by night on single plot

In [ ]:
#inset_axes?

In [ ]:
all_pwv = []
all_sigmapwv = []

# loop on nights
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_spec_night_wp = df_spec_wp[df_spec_wp["nightObs"] == night]
    df_spec_night_np = df_spec_np[df_spec_np["nightObs"] == night]
    df_spec_night_np_f = df_spec_np_f[df_spec_np_f["nightObs"] == night]
    df_spec_night_np_b = df_spec_np_b[df_spec_np_b["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["id","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_wp = df_spec_night_wp[["id","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_b = df_spec_night_np_b[["id","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_f = df_spec_night_np_f[["id","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np = df_spec_night_np[["id","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    list_of_targets = df_night_pwv_curve["TARGET"].unique()
    str_list_of_targets = "\n".join(list_of_targets)
    str_list_of_targets = "\n".join(["targets:",str_list_of_targets])
     

    # convert in hours
    #df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    #df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    #df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    #df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    #df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.


    #df_night_pwv_curve["dt"] = (pd.to_datetime(df_night_pwv_curve["Time"]) - tmin).dt.total_seconds()/3600.   
    #df_night_pwv_curve["dt"] = df_night_pwv_curve.apply(lambda row : (pd.to_datetime(row["Time"])-tmin).seconds,axis=1)
    df_night_pwv_curve.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_wp.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_f.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)

    #df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_b.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    #date_form = DateFormatter("%y-%m-%dT%H")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    N= len(df_night_pwv_curve)

    if N>10:
        # does the fit
        
        #x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
        xfit, y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean = MakeLinearModelFit(df_night_pwv_curve)
            
        sigma = np.std(resy)
        all_pwv.append(y_mean)
        all_sigmapwv.append(sigma )

        
        #textstr2 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",
        #             f"chi2/ndeg : {chi2dof:.2f}",       
        #            ))

        #textstr3 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",  
        #            ))
    

        #stat.loc["slope","PWV [mm]_x"] = slope 
        #stat.loc["slope_err","PWV [mm]_x"] = slope_err 
        #stat.loc["chi2","PWV [mm]_x"] = chi2dof

        ##########
        # plot
        ############
        
        fig, axs  = plt.subplots(2,2,figsize=(20,12),layout="constrained")
        ax,ax2,ax3,ax4 = axs.flatten()
        
        leg=ax.get_legend()
        
        ax.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",label="stat err",ms=1)
        df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="polar",ms=6,alpha=1)
        df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="bright, no polar",ms=6,alpha=1)
        df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="faint, no polar",ms=6,alpha=1)
        ax.set_xlabel("hours since midnight")
        ax.set_ylim(0.,15.)
        ax.set_ylabel("PWV (mm)")
        ax.legend(loc="upper right")

        ax.plot(xfit, y_ard,"g-",lw=3,label="ARDRegression")
        ax.fill_between(xfit, y_ard - 1.96 * y_ard_std,y_ard + 1.96 * y_ard_std,alpha=0.3,label=r"95% CL",facecolor='green')

        #ax.plot(xfit2, y_brr,"r-",label="BayesianRidge")
        #ax.fill_between(xfit2, y_brr - 1.96 * y_brr_std,y_brr + 1.96 * y_brr_std,alpha=0.5,label=r"95% CL",facecolor='red')
      
       
        ax.text(0.8, 0.5, str_list_of_targets, transform=ax.transAxes, fontsize=12,verticalalignment='top', bbox=props,alpha=0.5)
        #ax.set_title(f"night {night}")
        ax.axvline(x=0,color="k",lw=2)


        inset_ax = inset_axes(ax,width="20%", # width = 30% of parent_bbox
                                          height="40%", # height : 1 inch
                                          loc="upper left")
        
        textstr_sigma = f"$\sigma = $" +f"{sigma:.2f} mm"
        inset_ax.hist(resy,bins=30,facecolor="b")
        inset_ax.set_xlabel("residuals (mm)")
        inset_ax.text(0.1, 0.9, textstr_sigma , transform=inset_ax.transAxes, fontsize=8,verticalalignment='top', bbox=props,alpha=1)


        #all_figs_to_pdf.append(fig) 
       

        # Fetch the target SED, the spectra and plot
        # visits
        list_visitid = list(df_night_pwv_curve["id"])
        # target
        list_of_all_targets = np.unique(list(df_spec_night["TARGET"]))
        all_calspecs = {}
        for target_name in list_of_all_targets:
            c =  gcal.Calspec(target_name) 
            all_calspecs[target_name] = c.get_spectrum_numpy() 

        # smooth SED calspec
        all_calspecs_sm = {}
        span = 5
        for idx,target_name in enumerate(list_of_all_targets):
            c_dict = all_calspecs[target_name]
            wls = c_dict["WAVELENGTH"].value
            sed = c_dict["FLUX"].value
            sedm = smooth_data_np_convolve(sed,span)
            c_dict_sm = {"WAVELENGTH":wls,"FLUX":sedm}
            all_calspecs_sm[target_name] = c_dict_sm

        # get spectra
        all_foundvisitid, all_headers, all_params_spectrum , all_params_spectrogram, all_spectra = GetListOfSpectra(butler,list_visitid,my_collection)

        # do the plot
        #fig_spectra = plot_spectra(all_spectra, [spec.airmass for spec in all_spectra],collection=" ",dateobs=night,figsize=(12,6))
        plot_spectra_ax(all_spectra, ax2, [spec.airmass for spec in all_spectra],dateobs=night)
        #all_figs_to_pdf.append(fig_spectra) 
        #plt.show()

        #fig_transmission = plot_atmtransmission(all_spectra, [spec.airmass for spec in all_spectra],all_calspecs_sm,Telescope,Disp,collection=" ",dateobs=night,figsize=(12,6))
        plot_atmtransmission_ax(all_spectra,ax3 ,[spec.airmass for spec in all_spectra],all_calspecs_sm,Telescope,Disp,dateobs=night)
        
 
        #fig_transmission_zcorr = plot_atmtransmission_zcorr(all_spectra, [ spec.airmass for spec in all_spectra ],all_calspecs_sm,Telescope,Disp,collection=" " ,dateobs=night,figsize=(12,6) )
        plot_atmtransmission_zcorr_ax(all_spectra,ax4 ,[ spec.airmass for spec in all_spectra ],all_calspecs_sm,Telescope,Disp,dateobs=night )
        #all_figs_to_pdf.append(fig_transmission_zcorr) 
  
        suptitle =  f"night {night} , Nspec= {N}"
        plt.suptitle(suptitle,fontsize=20,fontweight="bold")

        # save figures
        figname =f"{pathfigs}/spectra_transmissions_{night}"+figtype
        plt.savefig(figname)
        
        all_figs_to_pdf.append(fig) 

        #save in hdf5
        #the_h5_file = savehdf5_pernightspectra(all_spectra,df_spec_night,all_calspecs_sm,Telescope,Disp,night,datapath_output)
        the_h5_file = savehdf5_pernightspectra_withsim(all_spectra,df_spec_night,all_calspecs_sm,Telescope,Disp,night,emul,datapath_output)
        
        plt.show()

## Fetch Spectra and transmission curve

### Select one night

In [ ]:
#night = all_selected_nights[-5]
night = 20240925

### make lists of night

In [ ]:
df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]

In [ ]:
tmin = df_spec_night["Time"].min()
tmax = df_spec_night["Time"].max()

list_of_targets = df_spec_night["TARGET"].unique()
str_list_of_targets = "\n".join(list_of_targets)
     
# convert in hours wrt midnight
df_spec_night.assign(dt = lambda row : (row["Time"]-tmin).dt.seconds/3600.,inplace=True)


### Make list of visitid

In [ ]:
list_visitid = list(df_spec_night["id"])

In [ ]:
list_of_all_targets = np.unique(list(df_spec_night["TARGET"]))

### Get Spectra

In [ ]:
all_foundvisitid, all_headers, all_params_spectrum , all_params_spectrogram, all_spectra = GetListOfSpectra(butler,list_visitid,my_collection)

In [ ]:
all_params_spectrum[-5]

In [ ]:
all_params_spectrogram[-5] 

In [ ]:
all_spectra[-5].data

### Get SED

In [ ]:
all_calspecs = {}
for target_name in list_of_all_targets:
    c =  gcal.Calspec(target_name) 
    all_calspecs[target_name] = c.get_spectrum_numpy() 

In [ ]:
all_calspecs_sm = {}
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot()
span = 5
fmax = np.zeros(len(list_of_all_targets))
for idx,target_name in enumerate(list_of_all_targets):
    c_dict = all_calspecs[target_name]
    wls = c_dict["WAVELENGTH"].value
    sed = c_dict["FLUX"].value
    ax.plot(wls,sed,'b-',label=target_name)

    fmax[idx] = sed[np.where(np.logical_and(wls>WLMIN*10,wls<WLMAX*10))[0]].max()
    
    sedm = smooth_data_np_convolve(sed,span)
    c_dict_sm = {"WAVELENGTH":wls,"FLUX":sedm}
    all_calspecs_sm[target_name] = c_dict_sm
    ax.plot(wls,sedm,'r-',label="smooth " +target_name)
ax.legend()
ax.set_xlim(3000.,10000.)   
ax.set_ylim(0.,fmax.max()*1.1)
ax.set_title(f"SEDs for night {night}")
ax.set_xlabel("$\\lambda (\\AA)$")

In [ ]:
count = 0
all_lists_of_targets = []
DATEOBS = night
N = len(all_spectra)
count += N
print(idx,DATEOBS,len(all_spectra),count) 
list_of_targets = np.unique([spec.target.label for spec in all_spectra])
    
if N>1:
    fig = plot_spectra(all_spectra, [spec.airmass for spec in all_spectra],collection=my_collection,dateobs=DATEOBS,figsize=(12,8))
else:
    idx= all_spectra[0].dataId
    print(f" \n /////// SKIPP {idx} \n")

In [ ]:
from libanaspectra import *
fig,ax = plt.subplots(1,1,figsize=(10,6))
plot_spectra_ax(all_spectra, ax, [spec.airmass for spec in all_spectra],dateobs=DATEOBS)

In [ ]:
if N > 1:
    plot_atmtransmission(all_spectra, [spec.airmass for spec in all_spectra],all_calspecs_sm,Telescope,Disp,collection=my_collection,dateobs=DATEOBS,figsize=(12,8))
else:
    id = filtered_spectra[0].dataId
    print(f" \n /////// SKIPP {id} \n")


## Plot atmospheric transmissions for airmass = 1

$$
T(z_{pred}) = \frac{ \left( T(z_{meas}) \right)^\left( \frac{z_{pred}}{z_{meas}}\right)}{(T^{grey}_{z_{meas}})^{z_{pred}}}
$$

In [ ]:
if N > 1:
    plot_atmtransmission_zcorr(all_spectra, [ spec.airmass for spec in all_spectra ],all_calspecs_sm,Telescope,Disp,collection=my_collection,dateobs=DATEOBS,figsize=(14,8) )
else:
    id = filtered_spectra[0].dataId
    print(f" \n /////// SKIPP {id} \n")    

## Save all figs in a pdf file

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
#pdf_filename = f"holo_PWV_night_fitvariation_andSpectra.pdf"
#pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
#with PdfPages(pdf_fullfilename) as pdf:
#    for fig in all_figs_to_pdf:
#        pdf.savefig(fig, bbox_inches='tight') 

In [ ]:
pdf_filename = f"holo_PWV_night_fitvariation_andSpectra_tight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig, bbox_inches='tight') 

In [ ]:
pdf_filename = f"holo_PWV_night_fitvariation_andSpectra_nottight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig) 

In [ ]:
df_night_pwv_curve